<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Trade-Statistics-of-Japan---Liquified-Natural-Gas" data-toc-modified-id="Trade-Statistics-of-Japan---Liquified-Natural-Gas-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Trade Statistics of Japan - Liquified Natural Gas</a></span></li></ul></div>

# Trade Statistics of Japan - Liquified Natural Gas 

In [1]:
import requests
import numpy as np
from bs4 import BeautifulSoup as bs
import pandas as pd
import html5lib
pd.options.display.float_format = '{:.4f}'.format
import xlrd

In [2]:
%%time
#Link to Japan customs data for LNG
# https://www.customs.go.jp/toukei/srch/indexe.htm?M=03&P=1,2,,,,,,,,1,0,2023,0,10,0,2,271111,,,,,,,,,,1,,,,,,,,,,,,,,,,,,,,,,200
# Dashboard: https://public.tableau.com/app/profile/gosia.skwara/viz/JapaneseLNGImports-pricesvolumevalue/Prices?publish=yes 

years = [*range(2017,2024)] #from 2020 to present
months = [*range(1,13)] #for month as number

all_dfs = []

for x in years:
    for y in months:
        try:
#         print(x, y)
            data = [
                ('CW_SEARCHID', 'JCCHT09S'), ('CW_JAPANKBN', '2'), ('CW_IMPKBN', '2'), ('CW_CARGOKBN', ''),
                ('CW_SUMKBN', ''), ('CW_SPCODE', ''), ('CW_SPNAME', ''), ('CW_YMSORTKBN', ''), ('CW_SISUKBN', ''),
                ('CW_SENKIKBN', ''), ('CW_HKKBN', ''), ('CW_YMKBN', '1'), ('CW_KI', ''), ('CW_SYY', str(x)),
                ('CW_EYY', ''), ('CW_SMM', str(y)), ('CW_EMM', ''), ('CW_HSKBN', '2'), ('CW_HSCODE', '271111'),
                ('CW_HSNAME', ''), ('CW_HSCODE', ''), ('CW_HSNAME', ''), ('CW_HSCODE', ''), ('CW_HSNAME', ''), 
                ('CW_HSCODE', ''), ('CW_HSNAME', ''), ('CW_HSCODE', ''), ('CW_HSNAME', ''), ('CW_HSCODE', ''),
                ('CW_HSNAME', ''), ('CW_HSCODE', ''), ('CW_HSNAME', ''), ('CW_HSCODE', ''), ('CW_HSNAME', ''),
                ('CW_HSCODE', ''), ('CW_HSNAME', ''), ('CW_HSCODE', ''), ('CW_HSNAME', ''), ('CW_KUNIKBN', '1'),
                ('CW_KUNICODE', ''), ('CW_KUNINAME', ''), ('CW_KUNICODE', ''), ('CW_KUNINAME', ''),
                ('CW_KUNICODE', ''), ('CW_KUNINAME', ''), ('CW_KUNICODE', ''), ('CW_KUNINAME', ''),
                ('CW_KUNICODE', ''), ('CW_KUNINAME', ''), ('CW_KUNICODE', ''), ('CW_KUNINAME', ''),
                ('CW_KUNICODE', ''), ('CW_KUNINAME', ''), ('CW_KUNICODE', ''), ('CW_KUNINAME', ''),
                ('CW_KUNICODE', ''), ('CW_KUNINAME', ''), ('CW_KUNICODE', ''), ('CW_KUNINAME', ''),
                ('CW_ZMKBN', ''), ('CW_ZMCODE', ''), ('CW_ZMNAME', ''), ('CW_ZMCODE', ''), ('CW_ZMNAME', ''),
                ('CW_ZMCODE', ''), ('CW_ZMNAME', ''), ('CW_ZMCODE', ''),('CW_ZMNAME', ''), ('CW_ZMCODE', ''),
                ('CW_ZMNAME', ''), ('CW_ZMCODE', ''), ('CW_ZMNAME', ''), ('CW_ZMCODE', ''), ('CW_ZMNAME', ''),
                ('CW_ZMCODE', ''), ('CW_ZMNAME', ''), ('CW_ZMCODE', ''), ('CW_ZMNAME', ''), ('CW_ZMCODE', ''),
                ('CW_ZMNAME', ''),('CW_MEISAICNT', '200')]

            r = requests.post('https://www.customs.go.jp/JCWSV03/servlet/JCWSV03', data=data) 
            df = pd.read_html(r.text)[2]
            df['date'] = str(x) + str(y)
            all_dfs.append(df) 
        
        except: 
            print("Not available for " + str(x) + str(y))

Not available for 202311
Not available for 202312
CPU times: user 9.2 s, sys: 248 ms, total: 9.45 s
Wall time: 1min 38s


In [5]:
df = pd.concat(all_dfs, ignore_index= True)
df = df.iloc[:, [0,4,5,9]] 
df = df.droplevel(0, axis = 1)
df = df.rename(columns = {'' : 'Date'})
df = df[df['CUSTOMS'].str.startswith('(')]
df['CUSTOMS'] = df['CUSTOMS'].str.replace('TOTAL', '').str.replace("\d\d\d.", '', regex = True)
df['CUSTOMS'] = df['CUSTOMS'].str.replace('(', '').str.replace(')', '')
df['Date'] = pd.to_datetime(df['Date'], format = '%Y%m')
df[['QUANTITY2', 'VALUE']] = df[['QUANTITY2', 'VALUE']].replace('-', np.nan) 
df[['QUANTITY2', 'VALUE']] = df[['QUANTITY2', 'VALUE']].apply(pd.to_numeric)
df = df.dropna()

# FXMJPY/CAD *only from 2017.... bs
r = requests.get('https://www.bankofcanada.ca/valet/observations/FXMJPYCAD/json?start_date=2017-01-01&end_date=2024-07-19')
df_fx_jpy = pd.json_normalize(r.json()['observations']) 
df_fx_jpy.columns = ['Date', 'FXMJPY/CAD']
df_fx_jpy['Date'] = pd.to_datetime(df_fx_jpy['Date'])
df_fx_jpy['FXMJPY/CAD'] = df_fx_jpy['FXMJPY/CAD'].apply(pd.to_numeric)

# FXMUSD/CAD
r = requests.get('https://www.bankofcanada.ca/valet/observations/FXMUSDCAD/json?start_date=2017-01-01&end_date=2024-07-19')
df_fx_usd = pd.json_normalize(r.json()['observations']) 
df_fx_usd.columns = ['Date', 'FXMUSD/CAD']
df_fx_usd['Date'] = pd.to_datetime(df_fx_usd['Date'])
df_fx_usd['FXMUSD/CAD'] = df_fx_usd['FXMUSD/CAD'].apply(pd.to_numeric)

# Brent 
url = 'https://www.eia.gov/dnav/pet/hist_xls/RBRTEm.xls'
df_brent = pd.read_excel(url, sheet_name = 'Data 1', skiprows=2)
df_brent = df_brent[(df_brent['Date'] > "2017-01-01")]
df_brent['Date'] = df_brent['Date'].astype(str)
df_brent['Date'] =  df_brent['Date'].str.replace('15','01') 
df_brent['Date'] = pd.to_datetime(df_brent['Date'], format = '%Y-%m-%d') 

# 1 barrel = 5.8 MMBtu https://www.bp.com/content/dam/bp/business-sites/en/global/corporate/pdfs/energy-economics/statistical-review/bp-stats-review-2022-approximate-conversion-factors.pdf
df_brent['Brent USD$/MMBtu'] = df_brent['Europe Brent Spot Price FOB (Dollars per Barrel)'] / 5.8

# merge everything
df_wfx = pd.merge(df, df_fx_jpy, how = 'left').merge(df_fx_usd).merge(df_brent)
df_wfx['VALUE'] = df_wfx['VALUE'] * 1000 

# FX conversion 
df_wfx['VALUE_CAD'] = df_wfx['VALUE'] * df_wfx['FXMJPY/CAD'] 
df_wfx['VALUE_USD'] = df_wfx['VALUE_CAD'] / df_wfx['FXMUSD/CAD']

# MT = Metric Tons (from https://www.customs.go.jp/english/tariff/abbreviation_of_unit.htm)
# 1 MT = 52 MMBTU (from https://www.spglobal.com/commodityinsights/PlattsContent/_assets/_files/en/conversiontables/conversion_factors.pdf
df_wfx['QUANTITY_MMBtu'] = df_wfx['QUANTITY2'] * 52 
df_wfx['USD$ per MMBtu'] = df_wfx['VALUE_USD'] / df_wfx['QUANTITY_MMBtu']

# to Bcf/d
df_wfx['QUANTITY_Bcf/d'] = df_wfx['QUANTITY_MMBtu'] / 365 /100000

#save to CSV
# df_wfx = df_wfx.iloc[:, [0,3,7,9,10,11, 12]] 
df_wfx.to_csv('JPY_LNG_prices.csv', index = False)


In [7]:
df_wfx.columns

Index(['CUSTOMS', 'QUANTITY2', 'VALUE', 'Date', 'FXMJPY/CAD', 'FXMUSD/CAD',
       'Europe Brent Spot Price FOB (Dollars per Barrel)', 'Brent USD$/MMBtu',
       'VALUE_CAD', 'VALUE_USD', 'QUANTITY_MMBtu', 'USD$ per MMBtu',
       'QUANTITY_Bcf/d'],
      dtype='object')

In [6]:
df_wfx.tail()

,CUSTOMS,QUANTITY2,VALUE,Date,FXMJPY/CAD,FXMUSD/CAD,Europe Brent Spot Price FOB (Dollars per Barrel),Brent USD$/MMBtu,VALUE_CAD,VALUE_USD,QUANTITY_MMBtu,USD$ per MMBtu,QUANTITY_Bcf/d
960,TURKEY,66607.0000,6643475000.0000,2023-10-01,0.0092,1.3717,90.6000,15.6207,60920665.7500,44412528.7964,3463564.0000,12.8228,0.0949
961,USA,567895.0000,49336507000.0000,2023-10-01,0.0092,1.3717,90.6000,15.6207,452415769.1900,329821221.2510,29530540.0000,11.1688,0.8091
962,NIGERIA,121618.0000,11122350000.0000,2023-10-01,0.0092,1.3717,90.6000,15.6207,101991949.5000,74354413.8660,6324136.0000,11.7572,0.1733
963,AUSTRAL,2397701.0000,221465655000.0000,2023-10-01,0.0092,1.3717,90.6000,15.6207,2030840056.3500,1480527853.2842,124680452.0000,11.8746,3.4159
964,PAP NGA,138587.0000,13233899000.0000,2023-10-01,0.0092,1.3717,90.6000,15.6207,121354853.8300,88470404.4835,7206524.0000,12.2764,0.1974
